In [1]:
import torch
import pandas as pd
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset 
BASE_DIR = "P:/pet ML"
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
X_tr = torch.load(f'{BASE_DIR}/data/tensor/X_tr.pt').to(device)
y_tr = torch.load(f'{BASE_DIR}/data/tensor/y_tr.pt').to(device)
X_test = torch.load(f'{BASE_DIR}/data/tensor/X_test.pt').to(device)
y_stats = torch.load(f'{BASE_DIR}/data/tensor/y_stats.pt')

In [3]:
input_feat = len(X_tr[0, :])

In [9]:
class model_nn(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(input_feat, 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.layer2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.layer3(x)
        return x

In [10]:
model = model_nn().to(device)
loss_fn = nn.MSELoss()
opt = Adam(model.parameters(), lr=0.005)
EPOCH = 50
n_iter = 100
train_ds = TensorDataset(X_tr, y_tr)
train_loader = DataLoader(dataset=train_ds, batch_size=256, shuffle=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, patience=10, factor=0.5)

In [12]:
model.train()
loss_list = []
for epoch in range(EPOCH):
    print(f'EPOCH {epoch+1} start')
    epoch_loss = 0
    for batch_X, batch_y in train_loader:
        opt.zero_grad()
        pred = model(batch_X)
        loss = loss_fn(pred.view(-1,1), batch_y)
        loss.backward()
        opt.step()
        epoch_loss += loss.item()
        
    epoch_loss /= len(train_loader)
    loss_list.append(epoch_loss)
    scheduler.step(epoch_loss)
    print(f'loss {epoch+1}: {epoch_loss:.6f}, lr: {opt.param_groups[0]["lr"]:.5f}')
    if (len(loss_list)) > 2:
        if loss_list[-2] - loss_list[-1] < 0.0001:
            print(f'EARLY STOPPING HERE!!!')
            break

EPOCH 1 start


c:\Users\polis\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\loss.py:626: UserWarning: Using a target size (torch.Size([176])) that is different to the input size (torch.Size([176, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss 1: 1.000181, lr: 0.00500
EPOCH 2 start
loss 2: 1.000160, lr: 0.00500
EPOCH 3 start
loss 3: 1.000160, lr: 0.00500
EARLY STOPPING HERE!!!


In [ ]:
model.eval()
pred_norm = model(X_test)
pred_real = pred_norm * y_stats['std'] + y_stats['mean']